# Import dependencies

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import requests
from census import Census

# Census API Key
from config import api_key
#change year as needed
c = Census(api_key, year=2016)

# SQLAlchemy
from sqlalchemy import create_engine

ModuleNotFoundError: No module named 'config'

# Kamran - Accidents from 2016-2018

# Store CSV into DataFrame

In [3]:
accident_csv_file = "Resources/US_Accidents_June20.csv"
accident_data_df = pd.read_csv(accident_csv_file)
accident_data_df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


# state_df

In [4]:
state_csv_file = "Resources/state_csvData.csv"
state_df = pd.read_csv(state_csv_file)
state_df.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [5]:
# Rename columns to match accident df for future merging
state_df = state_df.drop(columns={"Abbrev"})
state_df = state_df.rename(columns={"State": "state_name",
                                   "Code": "State"})
state_df.head()

,state_name,State
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


# accident_df

In [6]:
# view data before transformation
accident_data_df.nunique()

ID                       3513617
Source                         3
TMC                           21
Severity                       4
Start_Time               3200042
End_Time                 3246120
Start_Lat                1124695
Start_Lng                1113407
End_Lat                   375074
End_Lng                   383569
Distance(mi)               13476
Description              1780092
Number                     40365
Street                    176262
Side                           3
City                       11895
County                      1724
State                         49
Zipcode                   418780
Country                        1
Timezone                       4
Airport_Code                2001
Weather_Timestamp         546086
Temperature(F)               831
Wind_Chill(F)                974
Humidity(%)                  100
Pressure(in)                1022
Visibility(mi)                85
Wind_Direction                24
Wind_Speed(mph)              160
Precipitat

In [7]:
accident_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3513617 entries, 0 to 3513616
Data columns (total 49 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   TMC                    float64
 3   Severity               int64  
 4   Start_Time             object 
 5   End_Time               object 
 6   Start_Lat              float64
 7   Start_Lng              float64
 8   End_Lat                float64
 9   End_Lng                float64
 10  Distance(mi)           float64
 11  Description            object 
 12  Number                 float64
 13  Street                 object 
 14  Side                   object 
 15  City                   object 
 16  County                 object 
 17  State                  object 
 18  Zipcode                object 
 19  Country                object 
 20  Timezone               object 
 21  Airport_Code           object 
 22  Weather_Timestamp 

# Create new accident df

In [8]:
new_accident_data_df = accident_data_df[['ID', 'Start_Time', 'City', 'County', 'State', 'Zipcode', 'Street', 'Timezone', 'Temperature(F)', 'Visibility(mi)', 'Weather_Condition']].copy()
new_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy


In [9]:
# us_state_abbrev = {
#    'Alabama': 'AL',
#    'Alaska': 'AK',
#    'Arizona': 'AZ',
#    'Arkansas': 'AR',
#    'California': 'CA',
#    'Colorado': 'CO',
#    'Connecticut': 'CT',
#    'Delaware': 'DE',
#    'Florida': 'FL',
#    'Georgia': 'GA',
#    'Hawaii': 'HI',
#    'Idaho': 'ID',
#    'Illinois': 'IL',
#    'Indiana': 'IN',
#    'Iowa': 'IA',
#    'Kansas': 'KS',
#    'Kentucky': 'KY',
#    'Louisiana': 'LA',
#    'Maine': 'ME',
#    'Maryland': 'MD',
#    'Massachusetts': 'MA',
#    'Michigan': 'MI',
#    'Minnesota': 'MN',
#    'Mississippi': 'MS',
#    'Missouri': 'MO',
#    'Montana': 'MT',
#    'Nebraska': 'NE',
#    'Nevada': 'NV',
#    'New Hampshire': 'NH',
#    'New Jersey': 'NJ',
#    'New Mexico': 'NM',
#    'New York': 'NY',
#    'North Carolina': 'NC',
#    'North Dakota': 'ND',
#    'Ohio': 'OH',
#    'Oklahoma': 'OK',
#    'Oregon': 'OR',
#    'Pennsylvania': 'PA',
#    'Rhode Island': 'RI',
#    'South Carolina': 'SC',
#    'South Dakota': 'SD',
#    'Tennessee': 'TN',
#    'Texas': 'TX',
#    'Utah': 'UT',
#    'Vermont': 'VT',
#    'Virginia': 'VA',
#    'Washington': 'WA',
#    'West Virginia': 'WV',
#    'Wisconsin': 'WI',
#    'Wyoming': 'WY',
# }
# for key, value in us_state_abbrev.items():
#     new_accident_data_df.loc[new_accident_data_df['State']==key,'State'] = value
    
# new_accident_data_df = new_accident_data_df.append(value, ignore_index=True)
# new_accident_data_df.head()

# merge state and accident df

In [10]:
# Merge accident_df with state_df to obtain state_name for future merging with population df
clean_accident_data_df = pd.merge(new_accident_data_df, state_df, on= "State")
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast,Ohio
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy,Ohio
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy,Ohio


In [11]:
# convert start_time from object to datetime 
clean_accident_data_df['Start_Time'] = pd.to_datetime(clean_accident_data_df['Start_Time'])
clean_accident_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3513617 entries, 0 to 3513616
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ID                 object        
 1   Start_Time         datetime64[ns]
 2   City               object        
 3   County             object        
 4   State              object        
 5   Zipcode            object        
 6   Street             object        
 7   Timezone           object        
 8   Temperature(F)     float64       
 9   Visibility(mi)     float64       
 10  Weather_Condition  object        
 11  state_name         object        
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 348.5+ MB


In [12]:
clean_accident_data_df.nunique()

ID                   3513617
Start_Time           3200042
City                   11895
County                  1724
State                     49
Zipcode               418780
Street                176262
Timezone                   4
Temperature(F)           831
Visibility(mi)            85
Weather_Condition        127
state_name                49
dtype: int64

In [13]:
# Limit data from 2016-2020 to 2016 only
clean_accident_data_df = clean_accident_data_df[(clean_accident_data_df['Start_Time']>= "2016-01-01") & 
                                            (clean_accident_data_df['Start_Time']<= "2018-12-31")]
clean_accident_data_df.nunique()

ID                   2018197
Start_Time           1875832
City                   10793
County                  1658
State                     49
Zipcode               265911
Street                128158
Timezone                   4
Temperature(F)           780
Visibility(mi)            71
Weather_Condition         95
state_name                49
dtype: int64

In [14]:
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast,Ohio
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy,Ohio
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy,Ohio


In [15]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!! Didn't set ID as Index, ID is acting weird (ask team members) !!!!!!!!!!!!!!!!!!!!!!!!!!
clean_accident_data_df = clean_accident_data_df.sort_values(by='ID')
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
9,A-10,2016-02-08 08:10:04,Westerville,Franklin,OH,43081,Westerville Rd,US/Eastern,37.4,3.0,Light Rain,Ohio
99,A-100,2016-02-11 08:13:24,Dayton,Montgomery,OH,45410,US-35 E,US/Eastern,7.5,10.0,Scattered Clouds,Ohio
68791,A-1000,2016-06-23 10:31:12,El Dorado Hills,El Dorado,CA,95762,Latrobe Rd,US/Pacific,77.0,10.0,Clear,California
77791,A-10000,2017-01-06 16:22:04,West Sacramento,Yolo,CA,95691,I-80 W,US/Pacific,46.0,10.0,Clear,California


# Check to see if all 2016 data is present

In [16]:
# Sort by datetime
clean_accident_data_df = clean_accident_data_df.sort_values('Start_Time')
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain,Ohio
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,45203,I-75 S,US/Eastern,36.0,10.0,Overcast,Ohio


In [17]:
clean_accident_data_df.tail()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
1120468,A-3229937,2018-12-30 23:49:45,Orlando,Orange,FL,32832,Narcoossee Rd,US/Eastern,70.0,10.0,Scattered Clouds,Florida
1120467,A-3229936,2018-12-30 23:49:45,Orlando,Orange,FL,32832,Narcoossee Rd,US/Eastern,70.0,10.0,Scattered Clouds,Florida
3478703,A-3230370,2018-12-30 23:51:03,Wyoming,Chisago,MN,55092,I-35 N,US/Central,30.0,10.0,Overcast,Minnesota
2786921,A-3230568,2018-12-30 23:57:58,Astoria,Clatsop,OR,97103-2301,43rd St,US/Pacific,33.1,2.5,Clear,Oregon
1120485,A-3230429,2018-12-30 23:58:45,Defuniak Springs,Walton,FL,32435,I-10 E,US/Central,66.4,4.0,Overcast,Florida


In [18]:
clean_accident_data_df.count()

ID                   2018197
Start_Time           2018197
City                 2018133
County               2018197
State                2018197
Zipcode              2017659
Street               2018197
Timezone             2016487
Temperature(F)       1979434
Visibility(mi)       1971104
Weather_Condition    1970670
state_name           2018197
dtype: int64

In [19]:
clean_accident_data_df.nunique()

ID                   2018197
Start_Time           1875832
City                   10793
County                  1658
State                     49
Zipcode               265911
Street                128158
Timezone                   4
Temperature(F)           780
Visibility(mi)            71
Weather_Condition         95
state_name                49
dtype: int64

# Test to see if correct data is included

In [20]:
# clean_accident_data_df.loc[clean_accident_data_df['ID']=='A-9152']

In [21]:
# clean_accident_data_df.loc[clean_accident_data_df['ID']=='A-59918']

In [22]:
# clean_accident_data_df.loc[clean_accident_data_df['ID']=='A-9153']

In [23]:
# clean_accident_data_df.loc[clean_accident_data_df['Start_Time']=='2016-12-31 23:42:02']

In [24]:
clean_accident_data_df['normalised_date'] = clean_accident_data_df['Start_Time'].dt.normalize()
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name,normalised_date
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain,Ohio,2016-02-08
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio,2016-02-08
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio,2016-02-08
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio,2016-02-08
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,45203,I-75 S,US/Eastern,36.0,10.0,Overcast,Ohio,2016-02-08


In [25]:
clean_accident_data_df.loc[clean_accident_data_df['normalised_date']=='2018-12-30']

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name,normalised_date
59215,A-3229569,2018-12-30 00:00:03,Cincinnati,Hamilton,OH,45215,I-75 S,US/Eastern,35.1,10.0,Overcast,Ohio,2018-12-30
388708,A-1487670,2018-12-30 00:00:56,Santa Ana,Orange,CA,92701,I-5 N,US/Pacific,48.0,10.0,Clear,California,2018-12-30
2303361,A-3229570,2018-12-30 00:07:45,Aldie,Loudoun,VA,20105-1903,James Monroe Hwy,US/Eastern,33.8,10.0,Clear,Virginia,2018-12-30
388709,A-1487671,2018-12-30 00:07:49,Los Angeles,Los Angeles,CA,90012,W 3rd St,US/Pacific,46.0,10.0,Clear,California,2018-12-30
2712753,A-3230036,2018-12-30 00:10:00,Seattle,King,WA,98155,I-5 N,US/Pacific,44.1,10.0,Overcast,Washington,2018-12-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120468,A-3229937,2018-12-30 23:49:45,Orlando,Orange,FL,32832,Narcoossee Rd,US/Eastern,70.0,10.0,Scattered Clouds,Florida,2018-12-30
1120467,A-3229936,2018-12-30 23:49:45,Orlando,Orange,FL,32832,Narcoossee Rd,US/Eastern,70.0,10.0,Scattered Clouds,Florida,2018-12-30
3478703,A-3230370,2018-12-30 23:51:03,Wyoming,Chisago,MN,55092,I-35 N,US/Central,30.0,10.0,Overcast,Minnesota,2018-12-30
2786921,A-3230568,2018-12-30 23:57:58,Astoria,Clatsop,OR,97103-2301,43rd St,US/Pacific,33.1,2.5,Clear,Oregon,2018-12-30


# Reorganize df to look pretty

In [26]:
clean_accident_data_df = clean_accident_data_df[['ID', 'Start_Time', 'City', 'County', 'State', 'state_name', 'Zipcode', 'Street', 'Timezone', 'Temperature(F)', 'Visibility(mi)', 'Weather_Condition']]
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,state_name,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,Ohio,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,Ohio,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,Ohio,45203,I-75 S,US/Eastern,36.0,10.0,Overcast


In [27]:
# clean_accident_data_df.set_index("ID", inplace=True)

In [28]:
# clean_accident_data_df.head()

In [29]:
clean_accident_data_df = clean_accident_data_df.rename(columns={"State": "state_abbrev",
                                                               "state_name":"State",
                                                               "Start_Time": "Date_Time"})


# Tunde - Population from 2015-2018

In [30]:
#make API call to Census API for all US county estimated population for 2016
county_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "county:*"})

In [31]:
#2015
county_pop15_df = pd.DataFrame(county_pop_data)
county_pop15_df.rename(columns = {"B01001_001E":"2015"}, inplace=True)
county_pop15_df.head()

,NAME,2015,state,county
0,"Carroll County, Arkansas",27690,05,015
1,"Chicot County, Arkansas",11189,05,017
2,"Clark County, Arkansas",22684,05,019
3,"Clay County, Arkansas",15202,05,021
4,"Cleburne County, Arkansas",25587,05,023


In [32]:
#2016
county_pop16_df = pd.DataFrame(county_pop_data)
county_pop16_df.rename(columns = {"B01001_001E":"2016"}, inplace=True)
county_pop16_df.head()

,NAME,2016,state,county
0,"Carroll County, Arkansas",27690,05,015
1,"Chicot County, Arkansas",11189,05,017
2,"Clark County, Arkansas",22684,05,019
3,"Clay County, Arkansas",15202,05,021
4,"Cleburne County, Arkansas",25587,05,023


In [33]:
#2017
county_pop17_df = pd.DataFrame(county_pop_data)
county_pop17_df.rename(columns = {"B01001_001E":"2017"}, inplace=True)
county_pop17_df.head()

,NAME,2017,state,county
0,"Carroll County, Arkansas",27690,05,015
1,"Chicot County, Arkansas",11189,05,017
2,"Clark County, Arkansas",22684,05,019
3,"Clay County, Arkansas",15202,05,021
4,"Cleburne County, Arkansas",25587,05,023


In [34]:
#2018
county_pop18_df = pd.DataFrame(county_pop_data)
county_pop18_df.rename(columns = {"B01001_001E":"2018"}, inplace=True)
county_pop18_df.head()

,NAME,2018,state,county
0,"Carroll County, Arkansas",27690,05,015
1,"Chicot County, Arkansas",11189,05,017
2,"Clark County, Arkansas",22684,05,019
3,"Clay County, Arkansas",15202,05,021
4,"Cleburne County, Arkansas",25587,05,023


In [35]:
df_1516 = pd.merge(county_pop15_df, county_pop16_df, on="NAME")

In [36]:
df_1718 = pd.merge(county_pop17_df, county_pop18_df, on="NAME")

In [37]:
county_pop_df = pd.merge(df_1516, df_1718, on="NAME")

In [38]:
county_pop_df.head()

,NAME,2015,state_x_x,county_x_x,2016,state_y_x,county_y_x,2017,state_x_y,county_x_y,2018,state_y_y,county_y_y
0,"Carroll County, Arkansas",27690,05,015,27690,05,015,27690,05,015,27690,05,015
1,"Chicot County, Arkansas",11189,05,017,11189,05,017,11189,05,017,11189,05,017
2,"Clark County, Arkansas",22684,05,019,22684,05,019,22684,05,019,22684,05,019
3,"Clay County, Arkansas",15202,05,021,15202,05,021,15202,05,021,15202,05,021
4,"Cleburne County, Arkansas",25587,05,023,25587,05,023,25587,05,023,25587,05,023


In [39]:
county_pop_df.drop(columns=["state_x_x","county_x_x","state_y_x","county_y_x","state_x_y","county_x_y"], inplace=True)


In [40]:
county_pop_df.rename(columns = {"NAME":"county", "state_y_y":"state_id", "county_y_y":"county_id"}, inplace=True)

In [41]:
county_pop_df = county_pop_df[["county","county_id","state_id","2015","2016","2017","2018"]]
county_pop_df.head()

,county,county_id,state_id,2015,2016,2017,2018
0,"Carroll County, Arkansas",015,05,27690,27690,27690,27690
1,"Chicot County, Arkansas",017,05,11189,11189,11189,11189
2,"Clark County, Arkansas",019,05,22684,22684,22684,22684
3,"Clay County, Arkansas",021,05,15202,15202,15202,15202
4,"Cleburne County, Arkansas",023,05,25587,25587,25587,25587


In [42]:
county_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220 entries, 0 to 3219
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   county     3220 non-null   object
 1   county_id  3220 non-null   object
 2   state_id   3220 non-null   object
 3   2015       3220 non-null   object
 4   2016       3220 non-null   object
 5   2017       3220 non-null   object
 6   2018       3220 non-null   object
dtypes: object(7)
memory usage: 201.2+ KB


In [43]:
county_pop_df.count()

county       3220
county_id    3220
state_id     3220
2015         3220
2016         3220
2017         3220
2018         3220
dtype: int64

In [44]:
#make another call for state population just to get a list of states since county infor only have state census id
state_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "state:*"})

In [45]:
#pass on the state pop data to a df
state_pop_df = pd.DataFrame(state_pop_data)
state_pop_df.head()

,NAME,B01001_001E,state
0,Alabama,4841164,01
1,Alaska,736855,02
2,Arizona,6728577,04
3,Arkansas,2968472,05
4,California,38654206,06


In [46]:
#drop population column from the state pop data df
state_pop1_df = state_pop_df.drop(columns=["B01001_001E"])

In [47]:
state_pop1_df.head()

,NAME,state
0,Alabama,01
1,Alaska,02
2,Arizona,04
3,Arkansas,05
4,California,06


In [48]:
#rename "Name" column to prevent duplication when merging with county pop df
state_pop1_df.rename(columns = {"NAME":"state", "state":"state_id"}, inplace=True)

In [49]:
state_pop1_df.head()

,state,state_id
0,Alabama,01
1,Alaska,02
2,Arizona,04
3,Arkansas,05
4,California,06


In [50]:
#merge state and county df on the "state" the numeric identifier for states
county_pop1_df = pd.merge(county_pop_df, state_pop1_df, on="state_id")

In [51]:
county_pop1_df.head()

,county,county_id,state_id,2015,2016,2017,2018,state
0,"Carroll County, Arkansas",015,05,27690,27690,27690,27690,Arkansas
1,"Chicot County, Arkansas",017,05,11189,11189,11189,11189,Arkansas
2,"Clark County, Arkansas",019,05,22684,22684,22684,22684,Arkansas
3,"Clay County, Arkansas",021,05,15202,15202,15202,15202,Arkansas
4,"Cleburne County, Arkansas",023,05,25587,25587,25587,25587,Arkansas


In [52]:
#reorder columns 
county_pop1_df = county_pop1_df[["county","county_id","state","state_id","2015","2016","2017","2018"]]

In [53]:
county_pop1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220 entries, 0 to 3219
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   county     3220 non-null   object
 1   county_id  3220 non-null   object
 2   state      3220 non-null   object
 3   state_id   3220 non-null   object
 4   2015       3220 non-null   object
 5   2016       3220 non-null   object
 6   2017       3220 non-null   object
 7   2018       3220 non-null   object
dtypes: object(8)
memory usage: 226.4+ KB


In [54]:
#convert population columns to numeric
county_pop1_df[["2015","2016","2017","2018"]] = county_pop1_df[["2015","2016","2017","2018"]].apply(pd.to_numeric)

In [55]:
county_pop1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220 entries, 0 to 3219
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   county     3220 non-null   object
 1   county_id  3220 non-null   object
 2   state      3220 non-null   object
 3   state_id   3220 non-null   object
 4   2015       3220 non-null   int64 
 5   2016       3220 non-null   int64 
 6   2017       3220 non-null   int64 
 7   2018       3220 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 226.4+ KB


In [56]:
#import US state 2 letter abbreviation dictionary to create a state table 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [57]:
state_series = pd.Series(us_state_abbrev)

In [58]:
# print(state_series)

In [59]:
state_df = state_series.to_frame()

In [60]:
state_df.head()

,0
Alabama,AL
Alaska,AK
American Samoa,AS
Arizona,AZ
Arkansas,AR


In [61]:
state_df.reset_index(inplace=True)

In [62]:
state_df.head()

,index,0
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [63]:
state_df.rename(columns={"index":"state",0:"statecode"}, inplace=True)

In [64]:
state1_df = pd.merge(state_df,state_pop1_df, on="state" )

In [65]:
#County Estimated Population 2015 to 2018
county_pop1_df.head()

,county,county_id,state,state_id,2015,2016,2017,2018
0,"Carroll County, Arkansas",015,Arkansas,05,27690,27690,27690,27690
1,"Chicot County, Arkansas",017,Arkansas,05,11189,11189,11189,11189
2,"Clark County, Arkansas",019,Arkansas,05,22684,22684,22684,22684
3,"Clay County, Arkansas",021,Arkansas,05,15202,15202,15202,15202
4,"Cleburne County, Arkansas",023,Arkansas,05,25587,25587,25587,25587


In [66]:
#STATE TABLE
state1_df.head()

,state,statecode,state_id
0,Alabama,AL,01
1,Alaska,AK,02
2,Arizona,AZ,04
3,Arkansas,AR,05
4,California,CA,06


# Kamran edit population

In [67]:
county_pop1_df.rename(columns = {"2015":"2015 Population", 
                                "2016":"2016 Population",
                                "2017":"2017 Population",
                                "2018":"2018 Population"}, inplace=True)
county_pop1_df.head(300)

,county,county_id,state,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,"Carroll County, Arkansas",015,Arkansas,05,27690,27690,27690,27690
1,"Chicot County, Arkansas",017,Arkansas,05,11189,11189,11189,11189
2,"Clark County, Arkansas",019,Arkansas,05,22684,22684,22684,22684
3,"Clay County, Arkansas",021,Arkansas,05,15202,15202,15202,15202
4,"Cleburne County, Arkansas",023,Arkansas,05,25587,25587,25587,25587
...,...,...,...,...,...,...,...,...
295,"Pueblo County, Colorado",101,Colorado,08,162158,162158,162158,162158
296,"Rio Blanco County, Colorado",103,Colorado,08,6658,6658,6658,6658
297,"Rio Grande County, Colorado",105,Colorado,08,11623,11623,11623,11623
298,"Routt County, Colorado",107,Colorado,08,23980,23980,23980,23980


In [68]:
clean_pop_df = county_pop1_df.copy()
clean_pop_df.head()

,county,county_id,state,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,"Carroll County, Arkansas",015,Arkansas,05,27690,27690,27690,27690
1,"Chicot County, Arkansas",017,Arkansas,05,11189,11189,11189,11189
2,"Clark County, Arkansas",019,Arkansas,05,22684,22684,22684,22684
3,"Clay County, Arkansas",021,Arkansas,05,15202,15202,15202,15202
4,"Cleburne County, Arkansas",023,Arkansas,05,25587,25587,25587,25587


In [69]:
# Cleaned county column
clean_pop_df['county'] = clean_pop_df['county'].str.split(",", expand=True)[0]
clean_pop_df['county'] = clean_pop_df['county'].str.replace('County', '')
clean_pop_df.head()

,county,county_id,state,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,Carroll,015,Arkansas,05,27690,27690,27690,27690
1,Chicot,017,Arkansas,05,11189,11189,11189,11189
2,Clark,019,Arkansas,05,22684,22684,22684,22684
3,Clay,021,Arkansas,05,15202,15202,15202,15202
4,Cleburne,023,Arkansas,05,25587,25587,25587,25587


In [77]:
clean_pop_df.head(50)

,County,county_id,State,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,Carroll,015,Arkansas,05,27690,27690,27690,27690
1,Chicot,017,Arkansas,05,11189,11189,11189,11189
2,Clark,019,Arkansas,05,22684,22684,22684,22684
3,Clay,021,Arkansas,05,15202,15202,15202,15202
4,Cleburne,023,Arkansas,05,25587,25587,25587,25587
5,Cleveland,025,Arkansas,05,8421,8421,8421,8421
6,Columbia,027,Arkansas,05,24168,24168,24168,24168
7,Conway,029,Arkansas,05,21060,21060,21060,21060
8,Craighead,031,Arkansas,05,102884,102884,102884,102884
9,Crawford,033,Arkansas,05,61927,61927,61927,61927


In [71]:
# clean_pop_df = clean_pop_df.drop(columns={"state"})
clean_pop_df = clean_pop_df.rename(columns={"county": "County",
                                           "state": "State"})
clean_pop_df.head()

,County,county_id,State,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,Carroll,015,Arkansas,05,27690,27690,27690,27690
1,Chicot,017,Arkansas,05,11189,11189,11189,11189
2,Clark,019,Arkansas,05,22684,22684,22684,22684
3,Clay,021,Arkansas,05,15202,15202,15202,15202
4,Cleburne,023,Arkansas,05,25587,25587,25587,25587


# Merge accident and population

In [72]:
# clean_accident_data_df.head()

,ID,Date_Time,City,County,state_abbrev,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,Ohio,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,Ohio,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,Ohio,45203,I-75 S,US/Eastern,36.0,10.0,Overcast


In [78]:
# clean_accident_data_df.tail()

,ID,Date_Time,City,County,state_abbrev,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
1120468,A-3229937,2018-12-30 23:49:45,Orlando,Orange,FL,Florida,32832,Narcoossee Rd,US/Eastern,70.0,10.0,Scattered Clouds
1120467,A-3229936,2018-12-30 23:49:45,Orlando,Orange,FL,Florida,32832,Narcoossee Rd,US/Eastern,70.0,10.0,Scattered Clouds
3478703,A-3230370,2018-12-30 23:51:03,Wyoming,Chisago,MN,Minnesota,55092,I-35 N,US/Central,30.0,10.0,Overcast
2786921,A-3230568,2018-12-30 23:57:58,Astoria,Clatsop,OR,Oregon,97103-2301,43rd St,US/Pacific,33.1,2.5,Clear
1120485,A-3230429,2018-12-30 23:58:45,Defuniak Springs,Walton,FL,Florida,32435,I-10 E,US/Central,66.4,4.0,Overcast


In [73]:
# clean_accident_data_df.nunique()

ID                   2018197
Date_Time            1875832
City                   10793
County                  1658
state_abbrev              49
State                     49
Zipcode               265911
Street                128158
Timezone                   4
Temperature(F)           780
Visibility(mi)            71
Weather_Condition         95
dtype: int64

In [74]:
# clean_pop_df.nunique()

County             1955
county_id           325
State                52
state_id             52
2015 Population    3164
2016 Population    3164
2017 Population    3164
2018 Population    3164
dtype: int64

In [75]:
# Merge accident_df with pop_df
# clean_accident_data_df = pd.merge(new_accident_data_df, state_df, on= "State")
# clean_accident_data_df.head()

# accident_pop_df = pd.merge(clean_accident_data_df, clean_pop_df, on= "County", how='outer')
# accident_pop_df.head()

In [76]:
# accident_pop_merge_df = accident_pop_df.copy()
# accident_pop_merge_df.nunique()

NameError: name 'accident_pop_df' is not defined

In [ ]:
# accident_pop_merge_df = accident_pop_merge_df.sort_values('County')
# accident_pop_merge_df.tail(50)

In [ ]:
# # clean_accident_data_df.loc[clean_accident_data_df['ID']=='A-9152']
# accident_pop_merge_df.loc[accident_pop_merge_df['County']== 'Clark']

## Extract,Clean,Transform DL data 

In [3]:
#Extract Licensed Drivers by state (DL) csv into pandas DF for year 2014,2015,2016
DL_DF = pd.read_csv("Resources/DL.csv", usecols=[0,66,67,68])
DL_DF

,STATE,2014,2015,2016
0,Alabama,"3,881,542","3,907,038","3,943,082"
1,Alaska,"531,744","533,227","534,585"
2,Arizona,"4,881,801","4,978,762","5,082,305"
3,Arkansas,"2,111,873","2,119,578","2,391,103"
4,California,"24,813,346","25,532,920","26,199,436"
5,Colorado,"3,883,362","3,974,521","4,066,580"
6,Connecticut,"2,542,588","2,566,673","2,611,007"
7,Delaware,"732,349","742,524","756,328"
8,Dist. of Col.,"419,896","455,602","489,831"
9,Florida,"13,898,347","14,262,715","14,675,160"


In [4]:
# clean Licensed Drivers by state (DL)
DL_DF_Clean = DL_DF.rename(columns={'2014':'num_licenced_driver(2014)',
                                   '2015':'num_licenced_driver(2015)',
                                   '2016':'num_licenced_driver(2016)',
                                    'STATE':'state_name'})
DL_DF_Clean                       

,state_name,num_licenced_driver(2014),num_licenced_driver(2015),num_licenced_driver(2016)
0,Alabama,"3,881,542","3,907,038","3,943,082"
1,Alaska,"531,744","533,227","534,585"
2,Arizona,"4,881,801","4,978,762","5,082,305"
3,Arkansas,"2,111,873","2,119,578","2,391,103"
4,California,"24,813,346","25,532,920","26,199,436"
5,Colorado,"3,883,362","3,974,521","4,066,580"
6,Connecticut,"2,542,588","2,566,673","2,611,007"
7,Delaware,"732,349","742,524","756,328"
8,Dist. of Col.,"419,896","455,602","489,831"
9,Florida,"13,898,347","14,262,715","14,675,160"


## Extract,Clean,Transform RMV data 

In [5]:
#Extract Registered Motor Vehicles(RMV) csv into pandas DF
RMV_DF = pd.read_csv("Resources/RMV.csv", usecols=[0,15], skiprows=4)
RMV_DF

,Unnamed: 0,Unnamed: 15
0,Alabama,"5,468,301"
1,Alaska,"794,614"
2,Arizona,"5,786,891"
3,Arkansas,"2,808,138"
4,California,"30,221,033"
5,Colorado,"5,116,341"
6,Connecticut,"2,841,842"
7,Delaware,"1,003,840"
8,Dist. of Col.,"337,100"
9,Florida,"16,600,317"


In [6]:
# clean egistered Motor Veicheles(RMV)
RMV_DF_Clean = RMV_DF.rename(columns={'Unnamed: 0':'state_name','Unnamed: 15':'num_reg_vehicle'})
RMV_DF_Clean

,state_name,num_reg_vehicle
0,Alabama,"5,468,301"
1,Alaska,"794,614"
2,Arizona,"5,786,891"
3,Arkansas,"2,808,138"
4,California,"30,221,033"
5,Colorado,"5,116,341"
6,Connecticut,"2,841,842"
7,Delaware,"1,003,840"
8,Dist. of Col.,"337,100"
9,Florida,"16,600,317"
